In [29]:
import basicLib.loadAndTest as orig
import basicLib.featureUpdate as featUp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import *
from pylab import *
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import re
from sklearn.linear_model import LogisticRegression
import basicLib.modelRun as runms
from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time
import clf2net
import clf2netTest
import clf2netTree
from sklearn import ensemble

In [30]:
pathTrain = '../input_clean/users_2014_in_sessions.csv'
pathTest = '../input_clean/test_users.csv'

usersTrain = pd.read_csv(pathTrain)
usersTest = pd.read_csv(pathTest)
yRes = pd.DataFrame(usersTrain['country_destination'])
yFeat = featUp.countriesFeature()
yFeat.update(yRes)
usersTrain = usersTrain.drop('country_destination',axis=1)
users = pd.concat([usersTrain,usersTest],ignore_index=True)
featureStruct=featUp.generateFeatureStruct()
for feature in featureStruct:
    feature.update(users)
orig.initiateUsers(users)
featureList = orig.featureList(usersCol=users.columns)
Xtest = orig.sortFeatures(users,featureList.get())

In [38]:
Xlabel = Xtest
nonCategoryList=[]
for feature, dtype in Xlabel.dtypes.iteritems():
    if dtype.name != 'category':
        nonCategoryList.append(feature)
nonCategoryDF = Xlabel[nonCategoryList]
Xlabel = Xlabel.drop(nonCategoryList, 1)

labels = Xlabel.columns
myLabels = {} # holds the encode labels with value between 0 and n-1
n_values = [] # holds the length of each feature in bits ('gender' with 4 option will be 4 bits)
X = []
for label in Xlabel:
    labelAttr = getattr(Xlabel, label)
    myLabels[label] = preprocessing.LabelEncoder()
    myLabels[label].fit(labelAttr)
    if size(labelAttr.unique()) > 400:
        raise TypeError('The feature:', label, 'is too big to handle as binary')
    n_values.append(size(labelAttr.unique()))
    #transform: turn options into 0 to n-1,  reshape+transpose: align the coulumn so it be easier to add them to the table 
    X.append(myLabels[label].transform(labelAttr).reshape(1,len(labelAttr)).transpose())

#There is no category to change

temp = np.array([]).reshape(len(X[0]),0)
for i in range(len(X)):
    temp = concatenate((temp,X[i]),axis=1)
X = temp

enc = preprocessing.OneHotEncoder(n_values=n_values)
enc.fit(X)
X_byte = enc.transform(X).toarray()

X_byteDF = orig.makeDataFrame(X_byte,labels,n_values)
result = pd.concat([X_byteDF, nonCategoryDF], axis=1)

In [39]:
myDrop = {}
myList=('affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
        'first_browser', 'first_device_type', 'gender', 'language',
        'signup_app', 'signup_method', 'signup_flow')
for i in myList:
    attrTest = getattr(usersTest,i)
    attrTrain = getattr(usersTrain,i)
    myDrop[i] = []
    for x in attrTest.unique():
        if x not in attrTrain.unique():
            myDrop[i].append(x)
            print(i,':', x)
for key in myDrop.keys():
    for testCat in myDrop[key]:
        print(key,',', testCat)
        if pd.isnull(testCat): continue
        myInd = myLabels[key].classes_.tolist().index(testCat)
        label = key + '_' + str(myInd)
        result = result.drop(label,axis=1)

first_affiliate_tracked : nan
first_browser : Opera Mobile
first_browser : IBrowse
first_browser : Nintendo Browser
first_browser : CometBird
first_browser : UC Browser
language : -unknown-
signup_method : weibo
signup_flow : 14
signup_method , weibo
signup_flow , 14
language , -unknown-
first_affiliate_tracked , nan
first_browser , Opera Mobile
first_browser , IBrowse
first_browser , Nintendo Browser
first_browser , CometBird
first_browser , UC Browser


In [40]:
for i in result.columns:
    print(i)
len(result.columns)

affiliate_channel_0
affiliate_channel_1
affiliate_channel_2
affiliate_channel_3
affiliate_channel_4
affiliate_channel_5
affiliate_channel_6
affiliate_channel_7
affiliate_provider_0
affiliate_provider_1
affiliate_provider_2
affiliate_provider_3
affiliate_provider_4
affiliate_provider_5
affiliate_provider_6
affiliate_provider_7
affiliate_provider_8
affiliate_provider_9
affiliate_provider_10
affiliate_provider_11
affiliate_provider_12
affiliate_provider_13
affiliate_provider_14
affiliate_provider_15
affiliate_provider_16
first_affiliate_tracked_0
first_affiliate_tracked_1
first_affiliate_tracked_2
first_affiliate_tracked_3
first_affiliate_tracked_4
first_affiliate_tracked_5
first_affiliate_tracked_6
first_affiliate_tracked_7
first_browser_0
first_browser_1
first_browser_2
first_browser_3
first_browser_4
first_browser_5
first_browser_6
first_browser_7
first_browser_8
first_browser_10
first_browser_11
first_browser_12
first_browser_14
first_browser_15
first_browser_16
first_browser_17
first

137

In [41]:
result = orig.removeColumns(result,('date_account_created_month',))

In [5]:
result = result.drop('date_account_created_month_6',axis=1)
result = result.drop('date_account_created_month_7',axis=1)
result = result.drop('date_account_created_month_8',axis=1)

In [43]:
orig.normDf(result)
result['id'] = users['id']

In [44]:
finalTrain = result[0:len(yRes)]
finalTest = result[len(yRes):]

In [45]:
finalTrainUse = finalTrain.drop('id',axis=1)
finalTestUse = finalTest.drop('id',axis=1)

### training

In [8]:
X_train, X_test, y_train, y_test = train_test_split(finalTrain, yRes['country_destination'])

In [ ]:
predictMethod = LogisticRegression()
predictMethod.fit(X_train,y_train)

In [ ]:
y_pred = predictMethod.predict(X_test)
accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(X_test)

In [ ]:
accuracy_score

In [ ]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    myArr.append(predictMethod.classes_[order[:5]])

In [14]:
len(finalTrainUse)

213451

### send to airbnb - regression

In [46]:
start = time.time()
predictMethod = LogisticRegression()
predictMethod.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

76.1757378578186


In [47]:
y_pred = predictMethod.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(finalTestUse)

#### only 1 destionation

In [48]:
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}

In [49]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_regression_2014_woMonthes_S.csv',index=False)

#### to put 5 destination

In [50]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = usersTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)

In [51]:
d = {'id' : idCol , 'country' : pd.Series(myArr)}

In [52]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_regression_2014_woMonthes_M.csv',index=False)

### send to airbnb - tree

In [42]:
predictMethod_tree = tree.DecisionTreeClassifier()
predictMethod_tree.fit(finalTrainUse,yRes['country_destination'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [44]:
y_pred = predictMethod_tree.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod_tree.predict_proba(finalTestUse)

In [46]:
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}

In [47]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_tree_firstS.csv',index=False)

In [49]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = usersTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol , 'country' : pd.Series(myArr)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_tree_firstM.csv',index=False)

### send to airbnb - neural

In [17]:
start = time.time()
predictMethod_NN = MLPClassifier(solver='lbfgs', alpha=1e-5)
predictMethod_NN.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

53.58267402648926


In [18]:
y_pred = predictMethod_NN.predict(finalTestUse)
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod_NN.predict_proba(finalTestUse)
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_NN_2014S.csv',index=False)
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in predictMethod.classes_[order[:5]]:
        myArr.append(i)
idCol = usersTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol , 'country' : pd.Series(myArr)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_NN_2014M.csv',index=False)